Random Forest Implementation From Scratch
Dataset : http://archive.ics.uci.edu/ml/datasets/Wine
Practice Implementation from http://machinelearningmastery.com/implement-random-forest-scratch-python/

In [1]:
import pandas as pd
import numpy as np
wineDf = pd.read_csv(
    filepath_or_buffer='http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',
    header=None,
    sep=','
)
wineDf.columns = ['Label','Alcohol','Malic_acid','Ash','Alcalinity_of_ash','Magnesium','Total_phenols','Flavanoids','Nonflavanoid_phenols'
                  ,'Proanthocyanins','Color_intensity','Hue','OD280/OD315','Proline']

In [57]:
def get_split_node(random_sample_data, num_features):
    class_values = random_sample_data.ix[:,0]
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features_selected = np.random.randint(0, random_sample_data.shape[1], num_features)
    for index in features_selected:
        for row in range(random_sample_data.shape[0]):
            groups = test_split(index, random_sample_data[row:row+1].ix[:,index], random_sample_data)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

In [3]:
def cross_validation_split(dataset, num_folds):
    dataset_size = dataset.shape[0]
    size_folds = dataset_size/num_folds
    dataset_split = [dataset[(i*size_folds):((i+1)*size_folds)] for i in range(num_folds)]
    return dataset_split
    

In [4]:
def calculate_accuracy(actual, predicted):
    correct_predictions = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct_prediction +=1
    return correct_predictions/(float(len(actual))*100.0)

In [5]:
def evaluation(dataset, algorithm, num_folds, *args):
    folds = cross_validation_split(dataset, num_folds)
    scores = []
    for fold in folds:
        train_set = folds
        train_set.remove(fold)
        train_set = pd.concat(train_set)
        test_set = fold.ix[:,1:-1]
        predicted = algorithm(train_set, test_set, *args)
        actual = fold.ix[:,0]
        accuracy = calculate_accuracy(actual, predicted)
        scores.append(accuracy)
    return scores

In [6]:
def test_split(index, value, dataset):
    left = dataset[dataset.ix[:, index] < value]
    right = dataset[dataset.ix[:, index] >= value]
    return left, right
    

In [60]:
def gini_index(groups, class_values):
    gini_value = 0.0
    total_samples = left.shape[0] + right.shape[0]
    for label in class_values:
        left_label = left[left.ix[:,1] == label]
        size_left_label = left_label.shape[0]
        right_label = right[right.ix[:,1] == label]
        size_right_label = right_label.shape[0]
        temp_value = size_left_label/float(left.shape[0])
        label_value = (temp_value**2) + ((1 - temp_value)**2)
        gini_value+= (left.size/float(total_size))*label_value
    return gini_value

In [8]:
def to_terminal(group):
    outcomes = [row.ix[:,1] for row in group]
    return max(set(outcomes), key=outcomes.count)

In [9]:
def split(node, max_depth, min_size, num_features, depth):
    left, right = node['groups']
    del(node['groups'])
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        ssplit(node['left'], max_depth, min_size, n_features, depth+1)
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)

In [10]:
def build_tree(dataset, max_depth, min_size, num_features):
    root = get_split_node(dataset, num_features)
    split(root, max_depth, min_size, num_features, 1)
    return root

In [11]:
def predict(node, row):
    if row[node['index'] < node['value']]:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [52]:
def create_sample(dataset, sample_size):
    sample_set = []
    random_values = np.random.randint(0, wineDf.shape[0], 20)
    sample_set = []
    for value in random_values:
        sample_set.append(wineDf[value:value+1])
    sample_df = pd.concat(sample_set)
    return sample_df

In [13]:
def bagging_predictions(trees, row):
    predictions = [predict(node, row) for node in trees]
    return max(set(predictions), key=predictions.count)

In [14]:
def random_forest(train, test, max_depth, min_size, sample_size, num_trees, num_features):
    trees = []
    for i in range(num_trees):
        sample = create_sample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, num_features)
        trees.append(tree)
    predictions = [bagging_predictions(trees, row) for row in test]
    return predictions

In [62]:
# evaluate algorithm
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(np.sqrt(wineDf.shape[1])-1)
for n_trees in [1, 5, 10]:
	scores = evaluation(wineDf, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
	print('Trees: %d' % n_trees)
	print('Scores: %s' % scores)
	print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))